In [1]:
import os
import sys
import json
import argparse

import numpy as np
import matplotlib.pyplot as plt
import nibabel as nib

from glob import glob
from nilearn import plotting

In [2]:

# ### import data with `pybids` 
# based on: https://github.com/bids-standard/pybids/blob/master/examples/pybids_tutorial.ipynb
def import_bids_data(bidsroot, subject_id, task_label):
    from bids import BIDSLayout

    layout = BIDSLayout(bidsroot)

    all_files = layout.get()
    t1w_fpath = layout.get(return_type='filename', subject=subject_id, 
                            suffix='T1w', extension='nii.gz')[0]
    bold_files = layout.get(return_type='filename', subject=subject_id, 
                            suffix='bold', task=task_label, extension='nii.gz')
    return all_files, t1w_fpath, bold_files



In [3]:

# ## nilearn modeling: first level
# based on: https://nilearn.github.io/auto_examples/04_glm_first_level/plot_bids_features.html#sphx-glr-auto-examples-04-glm-first-level-plot-bids-features-py

def prep_models_and_args(subject_id=None, task_id=None, fwhm=None, bidsroot=None, 
                         deriv_dir=None, event_type=None, t_r=None, t_acq=None, space_label='T1w'):
    from nilearn.glm.first_level import first_level_from_bids
    from nilearn.interfaces.fmriprep import load_confounds

    data_dir = bidsroot

    task_label = task_id
    fwhm_sub = fwhm

    # correct the fmriprep-given slice reference (middle slice, or 0.5)
    # to account for sparse acquisition (silent gap during auditory presentation paradigm)
    # fmriprep is explicitly based on slice timings, while nilearn is based on t_r
    # and since images are only collected during a portion of the overall t_r (which includes the silent gap),
    # we need to account for this
    slice_time_ref = 0.5 * t_acq / t_r

    print(data_dir, task_label, space_label)

    models, models_run_imgs, models_events, models_confounds = first_level_from_bids(data_dir, task_label, space_label,
                                                                                     [subject_id],
                                                                                     smoothing_fwhm=fwhm,
                                                                                     derivatives_folder=deriv_dir,
                                                                                     slice_time_ref=slice_time_ref,
                                                                                     minimize_memory=False)

    # fill n/a with 0
    [[mc.fillna(0, inplace=True) for mc in sublist] for sublist in models_confounds]

    # define which confounds to keep as nuisance regressors
    conf_keep_list = ['framewise_displacement',
    #                #'a_comp_cor_00', 'a_comp_cor_01', 
    #                #'a_comp_cor_02', 'a_comp_cor_03', 
    #                #'a_comp_cor_04', 'a_comp_cor_05', 
    #                #'a_comp_cor_06', 'a_comp_cor_07', 
    #                #'a_comp_cor_08', 'a_comp_cor_09', 
                    'trans_x', 'trans_y', 'trans_z', 
                    'rot_x','rot_y', 'rot_z']

    ''' create events '''
    # stimulus events
    if event_type == 'stimulus':
        for sx, sub_events in enumerate(models_events):
            print(models[sx].subject_label)
            for mx, run_events in enumerate(sub_events):

                name_groups = run_events.groupby('trial_type')['trial_type']
                suffix = name_groups.cumcount() + 1
                repeats = name_groups.transform('size')

                run_events['trial_type'] = run_events['trial_type']
                run_events['trial_type'] = run_events['trial_type'].str.replace('-','_')

        # create stimulus list from updated events.tsv file
        stim_list = sorted([s for s in run_events['trial_type'].unique() if str(s) != 'nan'])
    
    # trial-specific events
    if event_type == 'trial':
        for sx, sub_events in enumerate(models_events):
            print(models[sx].subject_label)
            for mx, run_events in enumerate(sub_events):

                name_groups = run_events.groupby('trial_type')['trial_type']
                suffix = name_groups.cumcount() + 1
                repeats = name_groups.transform('size')

                run_events['trial_type'] = run_events['trial_type'] + \
                                                    '_trial' + suffix.map(str)
                run_events['trial_type'] = run_events['trial_type'].str.replace('-','_')

        # create stimulus list from updated events.tsv file
        stim_list = sorted([s for s in run_events['trial_type'].unique() if str(s) != 'nan'])

    # all sound events
    elif event_type == 'sound':
        for sx, sub_events in enumerate(models_events):
            print(models[sx].subject_label)
            for mx, run_events in enumerate(sub_events):
                orig_stim_list = sorted([str(s) for s in run_events['trial_type'].unique() if str(s) not in ['nan', 'None']])
                #print('original stim list: ', orig_stim_list)

                run_events['trial_type'] = run_events.trial_type.str.split('_', expand=True)[0]

        # create stimulus list from updated events.tsv file
        stim_list = sorted([str(s) for s in run_events['trial_type'].unique() if str(s) not in ['nan', 'None']])
        #print('stim list: ', stim_list)

    #model_and_args = zip(models, models_run_imgs, models_events, models_confounds)
    return stim_list, models, models_run_imgs, models_events, models_confounds, conf_keep_list



In [15]:

# ### Across-runs GLM
def nilearn_glm_across_runs(stim_list, task_label, models, models_run_imgs, \
                            models_events, models_confounds, conf_keep_list, space_label):
    from nilearn.reporting import make_glm_report
    for midx in range(len(models)):
        #for sx, stim in enumerate(stim_list):
        #contrast_label = 'fb_correct - fb_wrong' #stim
        #contrast_desc  = stim


        midx = 0
        model = models[midx]
        imgs = models_run_imgs[midx]
        events = models_events[midx]
        confounds = models_confounds[midx]

        print(model.subject_label)
        print(events[0])

        # set limited confounds
        print('selecting confounds')
        confounds_ltd = [models_confounds[midx][cx][conf_keep_list] for cx in range(len(models_confounds[midx]))]

        #try:
        # fit the GLM
        print('fitting GLM')
        model.fit(imgs, events, confounds_ltd); 
        print(model)

        # compute the contrast of interest
        print('computing contrast of interest')
        summary_statistics = model.compute_contrast('fb_correct - fb_wrong', output_type='all')
        zmap = summary_statistics['z_score']
        tmap = summary_statistics['stat']
        statmap = summary_statistics['effect_size']

        # get the residuals
        '''
        # save z map
        print('saving z-map')
        nilearn_sub_dir = os.path.join(bidsroot, 'derivatives', 'nilearn', 
                                    'level-1_fwhm-%.02f'%model.smoothing_fwhm, 
                                    'sub-%s_space-%s'%(model.subject_label, space_label),
                                    'run-all')
        if not os.path.exists(nilearn_sub_dir):
            os.makedirs(nilearn_sub_dir)

        analysis_prefix = 'sub-%s_task-%s_fwhm-%.02f_space-%s_contrast-%s'%(model.subject_label,
                                                                            task_label, model.smoothing_fwhm,
                                                                            space_label, contrast_desc)
        zmap_fpath = os.path.join(nilearn_sub_dir,
                                analysis_prefix+'_zmap.nii.gz')
        nib.save(zmap, zmap_fpath)
        print('saved z map to ', zmap_fpath)

        tmap_fpath = os.path.join(nilearn_sub_dir,
                                analysis_prefix+'_map-tstat.nii.gz')
        nib.save(tmap, tmap_fpath)
        print('saved t map to ', tmap_fpath)

        # also save beta maps
        statmap_fpath = os.path.join(nilearn_sub_dir,
                                    analysis_prefix+'_map-beta.nii.gz')
        nib.save(statmap, statmap_fpath)
        print('saved beta map to ', statmap_fpath)

        # save report
        print('saving report')
        report_fpath = os.path.join(nilearn_sub_dir,
                                    analysis_prefix+'_report.html')
        report = make_glm_report(model=model,
                                contrasts=contrast_label)
        report.save_as_html(report_fpath)
        print('saved report to ', report_fpath)
        '''
        #except:
        #    print('could not run for ', contrast_label)
    return zmap_fpath, statmap_fpath, contrast_label,


In [11]:

# ### Run-by-run GLM fit
def nilearn_glm_per_run(stim_list, task_label, event_filter, models, models_run_imgs, \
                        models_events, models_confounds, conf_keep_list, space_label):
    from nilearn.reporting import make_glm_report
    for midx in range(len(models)):
        stim_contrast_list = []
        for sx, stim in enumerate(stim_list):
            contrast_label = stim
            contrast_desc  = stim
            
            if event_filter in stim:
                print('running GLM with stimulus ', stim)

                midx = 0
                model = models[midx]
                imgs = models_run_imgs[midx]
                events = models_events[midx]
                confounds = models_confounds[midx]

                print(model.subject_label)

                # set limited confounds
                print('selecting confounds')
                confounds_ltd = [models_confounds[midx][cx][conf_keep_list] for cx in range(len(models_confounds[midx]))]

                for rx in range(len(imgs)):
                    img = imgs[rx]
                    event = events[rx]
                    confound = confounds_ltd[rx]

                    try:
                        # fit the GLM
                        print('fitting GLM on ', img)
                        model.fit(img, event, confound);

                        # compute the contrast of interest
                        print('computing contrast of interest', ' with contrast label = ', contrast_label)
                        statmap = model.compute_contrast(contrast_label, output_type='effect_size')
                        
                        # save stat map
                        print('saving beta map')
                        nilearn_sub_dir = os.path.join(bidsroot, 'derivatives', 'nilearn', 
                                                    'level-1_fwhm-%.02f'%model.smoothing_fwhm, 
                                                    'sub-%s_space-%s'%(model.subject_label, space_label))
                        if 'trial' in stim:
                            nilearn_sub_run_dir = os.path.join(nilearn_sub_dir, 'trial_models', 'run%02d'%rx)
                        else:
                            nilearn_sub_run_dir = os.path.join(nilearn_sub_dir, 'stimulus_per_run', 'run%02d'%rx)

                        if not os.path.exists(nilearn_sub_run_dir):
                            os.makedirs(nilearn_sub_run_dir)

                        analysis_prefix = ('sub-%s_task-%s_fwhm-%.02f_'
                                           'space-%s_contrast-%s_run%02d'%(model.subject_label,
                                                                           task_label, model.smoothing_fwhm,
                                                                           space_label, contrast_desc,
                                                                           rx))
                        statmap_fpath = os.path.join(nilearn_sub_run_dir,
                                                analysis_prefix+'_map-beta.nii.gz')
                        nib.save(statmap, statmap_fpath)
                        print('saved beta map to ', statmap_fpath)

                        stim_contrast_list.append(contrast_label)

                    except:
                        print('could not run for ', img, ' with ', contrast_label)

In [6]:
def plot_stat_maps(zmap, nilearn_dir, p_val=0.005):
    from scipy.stats import norm
    thresh_unc = norm.isf(p_val)

    # plot
    fig, axes = plt.subplots(3, 1, figsize=(20, 15))
    plotting.plot_stat_map(zmap, bg_img=t1w_fpath, colorbar=True, threshold=thresh_unc,
                        title='sub-%s %s (unc p<%.03f; fwhm=%.02f)'%(model.subject_label, 
                                                                    contrast_label ,p_val,fwhm_sub),
                        axes=axes[0],
                        display_mode='x', cut_coords=6)
    plotting.plot_stat_map(zmap, bg_img=t1w_fpath, colorbar=True, threshold=thresh_unc,
                        axes=axes[1],
                        display_mode='y', cut_coords=6)
    plotting.plot_stat_map(zmap, bg_img=t1w_fpath, colorbar=True, threshold=thresh_unc,
                        axes=axes[2],
                        display_mode='z', cut_coords=6)
    plotting.show()

    # save plot
    plot_fpath = os.path.join(nilearn_dir, 
                            'sub-%s_task-%s_fwhm-%.02f_pval-%.03f_space-%s_contrast-%s.png'%(model.subject_label,
                                                                        task_label,fwhm_sub, p_val,
                                                                        space_label, contrast_desc))
    fig.savefig(plot_fpath)

    return plot_fpath

In [7]:
# ## plot tsnr
def plot_tsnr(bold_files):
    from nilearn import image

    thresh = 0
    fwhm = 5

    for fx,filepath in enumerate(bold_files):
        tsnr_func = image.math_img('img.mean(axis=3) / img.std(axis=3)', img=filepath)
        tsnr_func_smooth = image.smooth_img(tsnr_func, fwhm=5)

        display = plotting.plot_stat_map(tsnr_func_smooth, 
                                        bg_img=t1w_fpath, 
                                        #title='fMRI single run tSNR map',
                                        #cut_coords=[8,50,-20],
                                        #threshold=thresh, 
                                        #cmap='jet'
                                        );
        display.show()
        

In [8]:
def mask_fmri(fmri_niimgs, mask_filename, fwhm):
    from nilearn.maskers import NiftiMasker
    masker = NiftiMasker(mask_img=mask_filename, 
                         smoothing_fwhm=fwhm, standardize=True,
                         memory="nilearn_cache", memory_level=1)
    fmri_masked = masker.fit_transform(fmri_niimgs)
    return fmri_masked, masker

## Run pipelines

In [9]:
task_label = 'tonecat'
t_acq = 2
t_r = 3

project_dir = os.path.join('/bgfs/bchandrasekaran/krs228/data/', 'FLT/')
bidsroot = os.path.join(project_dir,'data_bids')
#bidsroot = os.path.join(project_dir,'data_bids_noIntendedFor')
deriv_dir = os.path.join(project_dir, 'derivatives', 
                         '22.1.1', #'fmriprep_noSDC',
                        )

nilearn_dir = os.path.join(bidsroot, 'derivatives', 'nilearn')
if not os.path.exists(nilearn_dir):
        os.makedirs(nilearn_dir)

### Univariate analysis

Takes approximately 30 minutes per subject

In [10]:
fwhm = 4.5
space_label = 'MNI152NLin2009cAsym'
subject_list = ['FLT05', ] # 
# 'FLT01', 'FLT03', 'FLT04', 'FLT06', 'FLT07', 'FLT08', 'FLT09', 'FLT10', 'FLT11', 'FLT12', 'FLT13',

In [16]:
for sx, subject_id in enumerate(['FLT05']):
    print('Running subject ', subject_id)
    # Univariate analysis: MNI space, 3 mm, across-run GLM
    stim_list, models, models_run_imgs, \
        models_events, models_confounds, conf_keep_list = prep_models_and_args(subject_id, task_label, 
                                                                                 fwhm, bidsroot, 
                                                                                 deriv_dir, 'stimulus', #'sound',
                                                                                 t_r, t_acq, 
                                                                                 space_label)
   
    # Across-run GLM
    zmap_fpath, statmap_fpath, contrast_label = nilearn_glm_across_runs(stim_list, task_label, 
                                                                        models, models_run_imgs, 
                                                                        models_events, models_confounds, 
                                                                        conf_keep_list, space_label)


Running subject  FLT05
/bgfs/bchandrasekaran/krs228/data/FLT/data_bids tonecat MNI152NLin2009cAsym


/bgfs/bchandrasekaran/krs228/software/miniconda3/envs/py3/lib/python3.9/site-packages/nilearn/glm/first_level/first_level.py:948: UserWarning: SliceTimingRef not found in file /bgfs/bchandrasekaran/krs228/data/FLT/derivatives/22.1.1/sub-FLT02/func/sub-FLT02_task-tonecat_run-01_desc-preproc_bold.json. It will be assumed that the slice timing reference is 0.0 percent of the repetition time. If it is not the case it will need to be set manually in the generated list of models
  warn('SliceTimingRef not found in file %s. It will be assumed'


FLT05
FLT05
FLT05
     Unnamed: 0       onset  duration    trial_type  \
0             0    2.500000  0.300000  sound_di3_aN   
1             1    4.306723  0.500000        resp_6   
2             2    6.662803  0.749241      fb_wrong   
3             3    8.494559  0.300000  sound_di4_aN   
4             4    9.652414  0.500000        resp_2   
..          ...         ...       ...           ...   
139         139  350.984444  0.500000        resp_6   
140         140  353.437809  0.749006    fb_correct   
141         141  356.051873  0.300000  sound_di1_hN   
142         142  356.979323  0.500000        resp_7   
143         143  359.631972  0.749663    fb_correct   

                                   stim_file  response_time  correct_key  
0    stimuli/di3-aN_48000Hz_pol2_S15filt.wav            NaN          NaN  
1                                        NaN       1.806723          1.0  
2                                        NaN            NaN          NaN  
3    stimuli/di4-aN_4

/bgfs/bchandrasekaran/krs228/software/miniconda3/envs/py3/lib/python3.9/site-packages/nilearn/glm/first_level/experimental_paradigm.py:89: UserWarning: Unexpected column `correct_key` in events data will be ignored.
  warnings.warn(("Unexpected column `{}` in events "
/bgfs/bchandrasekaran/krs228/software/miniconda3/envs/py3/lib/python3.9/site-packages/nilearn/glm/first_level/experimental_paradigm.py:89: UserWarning: Unexpected column `response_time` in events data will be ignored.
  warnings.warn(("Unexpected column `{}` in events "
/bgfs/bchandrasekaran/krs228/software/miniconda3/envs/py3/lib/python3.9/site-packages/nilearn/glm/first_level/experimental_paradigm.py:89: UserWarning: Unexpected column `Unnamed: 0` in events data will be ignored.
  warnings.warn(("Unexpected column `{}` in events "
/bgfs/bchandrasekaran/krs228/software/miniconda3/envs/py3/lib/python3.9/site-packages/nilearn/glm/first_level/experimental_paradigm.py:89: UserWarning: Unexpected column `stim_file` in events 

FirstLevelModel(minimize_memory=False, slice_time_ref=0.3333333333333333,
                smoothing_fwhm=4.5, subject_label='FLT05', t_r=3.0)
computing contrast of interest


/bgfs/bchandrasekaran/krs228/software/miniconda3/envs/py3/lib/python3.9/site-packages/nilearn/glm/first_level/first_level.py:696: UserWarning: One contrast given, assuming it for all 6 runs
  warn('One contrast given, assuming it for all %d runs' % n_runs)


UndefinedVariableError: name 'fb_wrong' is not defined

#### test custom design matrix

In [18]:
len(models)

1

In [17]:
midx = 0
#for sx, stim in enumerate(stim_list):
#contrast_label = 'fb_correct - fb_wrong' #stim
#contrast_desc  = stim


midx = 0
model = models[midx]
imgs = models_run_imgs[midx]
events = models_events[midx]
confounds = models_confounds[midx]

print(model.subject_label)
print(events[0])

# set limited confounds
print('selecting confounds')
confounds_ltd = [models_confounds[midx][cx][conf_keep_list] for cx in range(len(models_confounds[midx]))]


FLT05
     Unnamed: 0       onset  duration    trial_type  \
0             0    2.500000  0.300000  sound_di3_aN   
1             1    4.306723  0.500000        resp_6   
2             2    6.662803  0.749241      fb_wrong   
3             3    8.494559  0.300000  sound_di4_aN   
4             4    9.652414  0.500000        resp_2   
..          ...         ...       ...           ...   
139         139  350.984444  0.500000        resp_6   
140         140  353.437809  0.749006    fb_correct   
141         141  356.051873  0.300000  sound_di1_hN   
142         142  356.979323  0.500000        resp_7   
143         143  359.631972  0.749663    fb_correct   

                                   stim_file  response_time  correct_key  
0    stimuli/di3-aN_48000Hz_pol2_S15filt.wav            NaN          NaN  
1                                        NaN       1.806723          1.0  
2                                        NaN            NaN          NaN  
3    stimuli/di4-aN_48000Hz_pol2_

In [19]:
from nilearn.glm.first_level import make_first_level_design_matrix

In [24]:
model.min_onset

-24

In [32]:
import nilearn
nilearn.__version__

'0.9.2'

In [23]:
stim_list

['fb_correct',
 'fb_wrong',
 'resp_1',
 'resp_2',
 'resp_6',
 'resp_7',
 'sound_di1_aN',
 'sound_di1_bN',
 'sound_di1_hN',
 'sound_di1_iN',
 'sound_di2_aN',
 'sound_di2_bN',
 'sound_di2_hN',
 'sound_di2_iN',
 'sound_di3_aN',
 'sound_di3_bN',
 'sound_di3_hN',
 'sound_di3_iN',
 'sound_di4_aN',
 'sound_di4_bN',
 'sound_di4_hN',
 'sound_di4_iN']

In [6]:
# just one subject
for sx, subject_id in enumerate([subject_list[0]]):
    print('Running subject ', subject_id)
    # Univariate analysis: MNI space, 3 mm, across-run GLM
    stim_list, models, models_run_imgs, \
        models_events, models_confounds, conf_keep_list = prep_models_and_args(subject_id, task_label, 
                                                                                 fwhm, bidsroot, 
                                                                                 deriv_dir, 'sound',
                                                                                 t_r, t_acq, 
                                                                                 space_label)

Running subject  FLT03


NameError: name 'task_label' is not defined

In [16]:
models

[FirstLevelModel(minimize_memory=False, slice_time_ref=0.3333333333333333,
                 smoothing_fwhm=4.5, subject_label='FLT02', t_r=3.0)]

#### test: residuals

In [12]:
subject_id = 'FLT08'

stim_list, models, models_run_imgs, models_events, \
        models_confounds, conf_keep_list = prep_models_and_args(subject_id, task_label, 
                                                                fwhm, bidsroot, 
                                                                deriv_dir, 'sound',
                                                                t_r, t_acq, 
                                                                space_label)

for midx in [0]: #range(len(models)):
    for sx, stim in enumerate(['sound']):
        contrast_label = stim
        contrast_desc  = stim


        midx = 0
        model = models[midx]
        imgs = models_run_imgs[midx]
        events = models_events[midx]
        confounds = models_confounds[midx]

        print(model.subject_label)

        # set limited confounds
        print('selecting confounds')
        confounds_ltd = [models_confounds[midx][cx][conf_keep_list] for cx in range(len(models_confounds[midx]))]

        # fit the GLM
        print('fitting GLM')
        model.fit(imgs, events, confounds_ltd);

/bgfs/bchandrasekaran/krs228/data/FLT/data_bids_noIntendedFor tonecat MNI152NLin2009cAsym
FLT08
FLT08
selecting confounds
fitting GLM


/bgfs/bchandrasekaran/krs228/software/miniconda3/envs/py3/lib/python3.9/site-packages/nilearn/glm/first_level/first_level.py:948: UserWarning: SliceTimingRef not found in file /bgfs/bchandrasekaran/krs228/data/FLT/derivatives/fmriprep_noSDC/sub-FLT01/func/sub-FLT01_task-tonecat_run-01_hemi-L_space-fsnative_bold.json. It will be assumed that the slice timing reference is 0.0 percent of the repetition time. If it is not the case it will need to be set manually in the generated list of models
  warn('SliceTimingRef not found in file %s. It will be assumed'
/bgfs/bchandrasekaran/krs228/software/miniconda3/envs/py3/lib/python3.9/site-packages/nilearn/glm/first_level/experimental_paradigm.py:89: UserWarning: Unexpected column `response_time` in events data will be ignored.
  warnings.warn(("Unexpected column `{}` in events "
/bgfs/bchandrasekaran/krs228/software/miniconda3/envs/py3/lib/python3.9/site-packages/nilearn/glm/first_level/experimental_paradigm.py:89: UserWarning: Unexpected column

In [13]:
model.residuals

### Multivariate analysis

Three options for multivariate analysis:
1. across-run stimulus modeling (n_output = n_stimuli)
2. within-run stimulus modeling (n_output = n_stimuli x n_runs)
3. within-run trial modeling (n_output = n_trials x n_runs)

In [ ]:
space_label = 'MNI152NLin2009cAsym'
subject_list = ['FLT01', 'FLT02', 'FLT03', 'FLT04', 'FLT05', 'FLT06', 'FLT07', 'FLT08', 'FLT09', 'FLT10', 'FLT11', 'FLT12', 'FLT13',]
fwhm_sub = 0

In [ ]:
print(subject_list)
print('analysis space: ', space_label)
print('task: ', task_label)
print('bids root directory: ', bidsroot)
print('derivatives directory: ', deriv_dir)

#### Stimulus modeling (across-run estimates)

In [ ]:
# Multivariate analysis: across-run GLM
event_type = 'stimulus'
event_filter = 'sound' 
for subject_id in subject_list:
    print('running with subject ', subject_id)

    stim_list, models, models_run_imgs, \
        models_events, models_confounds, conf_keep_list = prep_models_and_args(subject_id, task_label, fwhm_sub, bidsroot, 
                                                                               deriv_dir, event_type, t_r, t_acq, 
                                                                               space_label=space_label)
    print('stim list: ', stim_list)
    statmap_fpath, contrast_label = nilearn_glm_across_runs(stim_list, task_label, event_filter, models, models_run_imgs, 
                                                     models_events, models_confounds, conf_keep_list, space_label)
    #stat_maps, conditions = generate_conditions(subject_id, fwhm, space_label, deriv_dir)

#### Stimulus modeling: within-run GLM

In [ ]:
# Multivariate analysis: within-run GLM
event_type = 'stimulus'
event_filter = 'sound'
for subject_id in subject_list:
    print('running with subject ', subject_id)
        
    stim_list, models, models_run_imgs, \
        models_events, models_confounds, conf_keep_list = prep_models_and_args(subject_id, task_label, fwhm_sub, bidsroot, 
                                                                               deriv_dir, event_type, t_r, t_acq, 
                                                                               space_label=space_label)
    print('stim list: ', stim_list)
    statmap_fpath, contrast_label = nilearn_glm_per_run(stim_list, task_label, event_filter, models, models_run_imgs, 
                                                     models_events, models_confounds, conf_keep_list, space_label)
    #stat_maps, conditions = generate_conditions(subject_id, fwhm, space_label, deriv_dir)

#### Single trial modeling (within-run GLM)

Takes approximately 4–6 hours per subject

In [ ]:
# Multivariate analysis: within-run GLM
event_type = 'trial'
event_filter = 'sound'
for subject_id in subject_list:
    print('running with subject ', subject_id)
        
    stim_list, models, models_run_imgs, \
        models_events, models_confounds, conf_keep_list = prep_models_and_args(subject_id, task_label, fwhm_sub, bidsroot, 
                                                                               deriv_dir, event_type, t_r, t_acq, 
                                                                               space_label=space_label)
    print('stim list: ', stim_list)
    statmap_fpath, contrast_label = nilearn_glm_per_run(stim_list, task_label, event_filter, models, models_run_imgs, 
                                                     models_events, models_confounds, conf_keep_list, space_label)
    #stat_maps, conditions = generate_conditions(subject_id, fwhm, space_label, deriv_dir)

## tSNR estimates

In [ ]:
from nilearn import image
space_label = 'MNI152NLin2009cAsym'
fwhm = 1.5
tsnr_smooth = 3


In [ ]:
sub_list_mand = ['FLT01', 'FLT03', 'FLT05', 'FLT07', 'FLT08', 'FLT10', ] # 

sub_list_nman = ['FLT02', 'FLT04', 'FLT06', 'FLT09', 'FLT11', 'FLT12', 'FLT13', ]

sub_list = sub_list_mand + sub_list_nman

In [ ]:
roi_list = ['L-CN', 'L-SOC', 'L-IC', 'L-MGN', 'L-HG', 'L-PP', 'L-PT', 'L-STGp', 'L-STGa', 
            'R-CN', 'R-SOC', 'R-IC', 'R-MGN', 'R-HG', 'R-PP', 'R-PT', 'R-STGp', 'R-STGa', ]

In [ ]:
data_list_sub = []
data_list_roi = []
data_list_run = []
data_list_tsnr = []

for sx, subject_id in enumerate(sub_list):
    print('Running subject ', subject_id)
    # Univariate analysis: MNI space, 3 mm, across-run GLM
    stim_list, models, models_run_imgs, \
        models_events, models_confounds, conf_keep_list = prep_models_and_args(subject_id, task_label, 
                                                                                 fwhm, bidsroot, 
                                                                                 deriv_dir, 'sound',
                                                                                 t_r, t_acq, space_label)
    for mx, mask_descrip in enumerate(roi_list):
        #mask_descrip = 'L-STGp'
        print(mask_descrip)
        masks_dir = os.path.join(nilearn_dir, 'masks', 'sub-%s'%subject_id, 'space-%s'%space_label) #'masks-aparc' # 'masks-dseg' , 'masks-subcort-aud'
        mask_fpath = glob(masks_dir + '/*/' + 'sub-{}_space-{}_mask-{}.nii.gz'.format(subject_id, space_label, mask_descrip))[0]

        for px, preproc_fpath in enumerate(models_run_imgs[0]):
            tsnr_func = image.math_img('img.mean(axis=3) / img.std(axis=3)', img=preproc_fpath);
            tsnr_func_smooth = image.smooth_img(tsnr_func, fwhm=tsnr_smooth)

            #display = plotting.plot_epi(tsnr_func_smooth, colorbar=True, vmin=10, vmax=100,
            #                            title=os.path.basename(preproc_fpath));

            fmri_masked, masker = mask_fmri(tsnr_func_smooth, mask_fpath, fwhm)
            print(np.mean(fmri_masked))

            data_list_sub.append(subject_id)
            data_list_roi.append(mask_descrip)
            data_list_run.append(px+1)
            data_list_tsnr.append(np.mean(fmri_masked))

In [ ]:
print(len(data_list_sub))
print(len(data_list_roi))
print(len(data_list_run))
print(len(data_list_tsnr))


In [ ]:
np.unique(data_list_roi)

In [ ]:
np.unique(data_list_sub)

In [ ]:
import pandas as pd
import seaborn as sns

In [ ]:
tsnr_df = pd.DataFrame({'sub_id': data_list_sub, 'Region': data_list_roi, 'run': data_list_run, 'tSNR': data_list_tsnr})
tsnr_df['hemi'] = tsnr_df.Region.str[0]
tsnr_df.Region = tsnr_df.Region.str[2:]

In [ ]:
tsnr_df

In [ ]:
mean_tsnr_df = tsnr_df.groupby(['sub_id', 'Region', 'hemi'], as_index=False, sort=False)['tSNR'].mean()

In [ ]:
mean_tsnr_df.columns

In [ ]:
mean_tsnr_df

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(8,4), dpi=300)
sns.swarmplot(data=tsnr_df, x='Region', y='tSNR', hue='hemi')
fig.suptitle('temporal SNR by region of interest')

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(8,4), dpi=300)
sns.violinplot(data=mean_tsnr_df, x='Region', y='tSNR', hue='hemi')
fig.suptitle('temporal SNR by region of interest')